# **Mini-project \#2 - RegEx Tokenizer**

Name: \<please supply your name\>

More information on the assessment is found in our Canvas course.

# **Loading Data into Memory**

This code block is provided and helps load in the data for this assessment from the provided file. Specifically, we are loading in tweets into a Dataframe. The file contains two columns: (1) text and (2) created_at. The 2nd column isn't very useful here and is only really meant to give you an idea of the timeframe these tweets were created. Focus on the 1st columns, but should the time frame help you in your analysis, you're free to analyze that. Also, this code assumes you've placed the tweet csv into the folder where the Python notebook is located.

**Note**: Tweets collected here are public tweets. The collection of the tweets was done so through Twitter's API.

In [ ]:
import pandas as pd
import csv

tweets = pd.read_csv(
  "tweets_for_mp2.csv",
  index_col=False,
  quoting=csv.QUOTE_ALL)

In [ ]:
tweets.head()

# **Your Solution**

Kindly place your solution in the code block below. Make sure that aside from loading in and tokenizing the data, your code should also display the total tokens, total vocabulary, and list top 25 tokens with their repsective counts.